# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
city_weather = pd.read_csv("../output.csv")

city_weather.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=g_key)


In [ ]:
location = city_weather[["Latitude", "Longitude"]]
weight = city_weather["Humidity"]

#function for heatmap
def heatmap(locations, weights):
    weather_figure = gmaps.figure(center=(20,0), zoom_level=1.7)
    heat_layer = gmaps.heatmap_layer(location, weights=weight)
    weather_figure.add_layer(heat_layer)
    return weather_figure

#heatmap
humidity_heatmap = heatmap(location, weight)
humidity_heatmap

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#drop rows with null values
vacation_destinations = city_weather.dropna()

#filter out cities with undesirable weather conditions
vacation_destinations = vacation_destinations.drop(vacation_destinations.loc[vacation_destinations["Max Temp (C)"] < 65].index)

vacation_destinations = vacation_destinations.drop(vacation_destinations.loc[vacation_destinations["Max Temp (C)"] > 85].index)

vacation_destinations = vacation_destinations.drop(vacation_destinations.loc[vacation_destinations["Wind Speed"] > 10].index)

vacation_destinations = vacation_destinations.drop(vacation_destinations.loc[vacation_destinations["Cloudiness"] > 0].index)

vacation_destinations = vacation_destinations.drop(vacation_destinations.loc[vacation_destinations["Humidity"] > 60].index)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = vacation_destinations

In [ ]:
# list for all the responses
responses = []

#set parameters
target_type = "lodging"
radius = 5000

#API requests
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    coords = f"{lat},{lng}"
    params = {
        "location": coords,
        "types": target_type,
        "radius": radius,
        "key": gkey
    }
    response = requests.get(base_url, params).json()
    responses.append(response)

In [ ]:
#create an empty list for each variable
hotel_names = []
hotel_addresses = []
hotel_ratings = []
hotel_lats = []
hotel_lngs = []

for response in responses:
    hotel_names.append(response["results"][0]["name"])
    hotel_addresses.append(response["results"][0]["vicinity"])
    hotel_ratings.append(response["results"][0]["rating"])
    hotel_lats.append(response["results"][0]["geometry"]["location"]["lat"])
    hotel_lngs.append(response["results"][0]["geometry"]["location"]["lng"]) 

#Create new columns for the new data
hotel_df["Hotel Name"] = hotel_names
hotel_df["Hotel Address"] = hotel_addresses
hotel_df["Hotel Rating"] = hotel_ratings
hotel_df["Hotel Lat"] = hotel_lats
hotel_df["Hotel Lng"] = hotel_lngs


In [ ]:
#function for the map
def hotel_map(fig, locations, info_box_content):
    marker_layer = gmaps.marker_layer(locations, info_box_content=info_box_content)
    fig.add_layer(marker_layer)
    return fig

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer on top of heat map
fig = humidity_heatmap
hotel_locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

# Display Map
final_map = hotel_map(fig, hotel_locations, hotel_info)
final_map